In [1]:
import time
import re
import pylab as pl
import numpy as np
import pylab as pl

from datetime import datetime, timedelta

import subprocess

import os
import pandas as pd

import concurrent.futures


from joblib import Parallel, delayed
import shutil


from concurrent.futures import ThreadPoolExecutor, ProcessPoolExecutor
import subprocess

In [2]:
os.system('cd ../../; git add .; git commit -m "test"; git push -f origin HEAD:main')


On branch main
Your branch is up to date with 'origin/main'.

nothing to commit, working tree clean


Everything up-to-date


0

In [3]:
# result = subprocess.run(command, shell=True, capture_output=True, text=True)

# if result.returncode == 0:
#     print("Operation successful!")
# else:
#     print(f"Error: {result.stderr}")

In [4]:
# time.sleep(80)

In [5]:
outputs = []

In [6]:
# config_list = ['localn_12cls']
# config_list = ['cre_setup_2_scale_1_node_info']
# config_list = ['rvc']


# config_list = ['localn_12cls']

config_list = ['test48']
# config_list = ['test', 'test2']

# config_list2 = [\
#                 'local_12cls',\
# #                'global_3cls',\
# #                'global_4cls',\
# #                'global_6cls',\
#                'global_12cls',\
# #                'global_8cls',\
# #                'global_2cls',\
# #                'local_3cls',\
# #                'local_4cls',\
# #                'local_6cls',\
# #                'local_8cls',\
# #                'local_2cls'\
#                ]



config_list = config_list
# config_list = ['cre_setup_3_scale_1_node_info']




In [7]:
config_list

['test48']

In [8]:
cfig = config_list[0]
os.system('rm node_info.csv')
os.system('cp node_setup/'+cfig+'.csv node_info.csv')


0

In [9]:
#         os.system('gcloud compute instances delete --zone=europe-west3-c --quiet $(gcloud compute instances list --format="value(name)")')
#         os.system('gcloud compute instances delete --zone=asia-south1-c --quiet $(gcloud compute instances list --format="value(name)")')
#         os.system('gcloud compute instances delete --zone=us-west1-b --quiet $(gcloud compute instances list --format="value(name)")')   




In [10]:
run_opt = 'n' #input('Enter mode (r or fresh from scratch default)')

In [11]:
# time.sleep(120)

def str2datetime(s):
    parts = s.split('.')
    dt = datetime.strptime(parts[0], "%Y-%m-%d %H:%M:%S")
    return dt.replace(microsecond=int(parts[1]))


def remove_outliers(x, outlierConstant = 1.5):
    a = np.array(x)
    upper_quartile = np.percentile(a, 75)
    lower_quartile = np.percentile(a, 25)
    IQR = (upper_quartile - lower_quartile) * outlierConstant
    quartileSet = (lower_quartile - IQR, upper_quartile + IQR)
    resultList = []
    removedList = []
    for y in a.tolist():
        if y >= quartileSet[0] and y <= quartileSet[1]:
            resultList.append(y)
        else:
            removedList.append(y)
    return (resultList, removedList)
def getThroughput(experiment):

    result = subprocess.check_output('cat ../'+experiment+'/nohup_client_*', shell=True)


    result = result.decode("utf-8") 

    lines = result.split('\n')

    commit_pat = re.compile('([^[].*) \[hotstuff info\] ([0-9.]*)$')
#     commit_pat = re.compile('([^[].*) \[hotstuff info\] got*')

    Times = []
    lats = []

    for line in lines:
        m = commit_pat.match(line)
        if m:
            try:

                time_item = float((line.split(' ')[1]).split(':')[0])*3600 + float((line.split(' ')[1]).split(':')[1])*60 + \
                float((line.split(' ')[1]).split(':')[2])
                Times.append(time_item)        

                lats.append(float(m.group(2)))
            except:
                continue


    begin_time = None
    values = []
    interval = 1
    cnt = 0
    Times.sort()
    for timestamp in Times:
        if begin_time is None:
            begin_time = timestamp
            next_begin_time = timestamp +interval

        while timestamp >= next_begin_time:
            begin_time = next_begin_time
            next_begin_time += interval
            values.append(cnt)
            cnt = 0
        cnt += 1
    values.append(cnt)

#     lats, _ = remove_outliers(lats)
#     print(len(values))


    return np.arange(len(values))*interval,np.array(values)/interval, lats

# p = subprocess.Popen(['../../hotstuff-keygen', ' --num', '--n', str(4)],
#                      stdout=subprocess.PIPE, stderr=open(os.devnull, 'w'))

# print([[t[4:] for t in l.decode('ascii').split()] for l in p.stdout])
# # [[t for t in l.decode('ascii').split()] for l in p.stdout.read()]

# os.system('echo 3108 | sudo -S sh job_mini.sh '+ 'test')


# regions= ['us-west-1', 'asia-south1-c']
# regions= ['asia-south1-c', 'eu-central-1', 'us-west-2']
# regions= ['asia-south1-c', 'eu-central-1']
regions= ['us-west1-b']


instance_type = 't2.small'

df =pd.read_csv('node_info.csv')

node_info_dict = df.to_dict()

print(node_info_dict)



# with open("cluster_info_hs.txt", 'w') as file:

#     for key in node_info_dict['cluster_id']:
#         print(key, node_info_dict['cluster_id'][key])

#         if node_info_dict['cluster_id'][key] >= 0:
#             file.write(str(node_info_dict['cluster_id'][key]) + '\n')



lines  = []



for idx, key in enumerate( node_info_dict['region'].keys()):
    cmd =  'gcloud compute instances create resdb-'+f"{idx:03}"+' --project=ucr-ursa-major-lesani-lab --zone='+node_info_dict['region'][key].strip()[1:-1]+' --machine-type=e2-standard-2 --network-interface=network-tier=PREMIUM,stack-type=IPV4_ONLY,subnet=default --can-ip-forward --maintenance-policy=MIGRATE --provisioning-model=STANDARD --service-account=961693926925-compute@developer.gserviceaccount.com --scopes=https://www.googleapis.com/auth/devstorage.read_only,https://www.googleapis.com/auth/logging.write,https://www.googleapis.com/auth/monitoring.write,https://www.googleapis.com/auth/servicecontrol,https://www.googleapis.com/auth/service.management.readonly,https://www.googleapis.com/auth/trace.append --tags=http-server,https-server --create-disk=auto-delete=yes,boot=yes,device-name=instance-3,image=projects/ubuntu-os-cloud/global/images/ubuntu-2004-focal-v20230831,mode=rw,size=10,type=projects/ucr-ursa-major-lesani-lab/zones/us-west1-b/diskTypes/pd-balanced --no-shielded-secure-boot --shielded-vtpm --shielded-integrity-monitoring --labels=goog-ec-src=vm_add-gcloud --reservation-affinity=any'
#     cmd = 'gcloud compute instances create resdb-'+f"{lim:02}"+' --project=ucr-ursa-major-lesani-lab --zone='+reg+' --machine-type=e2-small --network-interface=network-tier=PREMIUM,stack-type=IPV4_ONLY,subnet=default --can-ip-forward --maintenance-policy=MIGRATE --provisioning-model=STANDARD --service-account=961693926925-compute@developer.gserviceaccount.com --scopes=https://www.googleapis.com/auth/devstorage.read_only,https://www.googleapis.com/auth/logging.write,https://www.googleapis.com/auth/monitoring.write,https://www.googleapis.com/auth/servicecontrol,https://www.googleapis.com/auth/service.management.readonly,https://www.googleapis.com/auth/trace.append --tags=http-server,https-server --create-disk=auto-delete=yes,boot=yes,device-name=instance-3,image=projects/ubuntu-os-cloud/global/images/ubuntu-2004-focal-v20230831,mode=rw,size=10,type=projects/ucr-ursa-major-lesani-lab/zones/us-west1-b/diskTypes/pd-balanced --no-shielded-secure-boot --shielded-vtpm --shielded-integrity-monitoring --labels=goog-ec-src=vm_add-gcloud --reservation-affinity=any'

    lines.append(cmd)


lines


def run_command(command):
    subprocess.call(command, shell=True)

commands = lines

# Create a ThreadPoolExecutor or ProcessPoolExecutor, depending on your needs
with concurrent.futures.ThreadPoolExecutor(max_workers=60) as executor:
    # Submit each command to the executor for parallel execution
    futures = [executor.submit(run_command, command) for command in commands]

    # Wait for all tasks to complete
#     concurrent.futures.wait(futures)

# time.sleep(60)

instance_type = 'e2-small'


redirect = '>'

os.system('gcloud compute instances list  --format="value(networkInterfaces[0].networkIP)" '+\
          redirect+' all_internal_ips')



os.system('gcloud compute instances list --format="value(networkInterfaces[0].networkIP)" '+\
      redirect+' all_external_ips')







!cat all_internal_ips
! echo " " 
# !cat all_external_ips



f = open('all_external_ips', "r")
data1 = [line.strip() for line in f.readlines()]

data1 = [x for x in data1 if x!='None']

f = open('all_internal_ips', "r")
data2 = [line.strip() for line in f.readlines()]

data2 = [x for x in data2 if x!='None']



data2, len(data2)

data = []

# for i in range(len(data1)):
#     if data1[i]!='None':
# #         data.append(data1[i] + '    ' + data2[i])
#         data.append('127.0.0.1'+ '    ' + '127.0.0.1')


skip_instances = 0

for i in range(skip_instances, len(data1)):
    if data1[i]!='None':
        data.append(data1[i] + '    ' + data2[i])
#         data.append('127.0.0.1'+ '    ' + '127.0.0.1')

data, len(data)

for index, item in enumerate(data1):
    if 'publicIP' not in node_info_dict.keys():
        node_info_dict['publicIP'] = {}


    node_info_dict['publicIP'][index] = item 


for index, item in enumerate(data2):
    if 'privateIP' not in node_info_dict.keys():
        node_info_dict['privateIP'] = {}

    node_info_dict['privateIP'][index] = item 



for index, item in enumerate(data):
    if 'publicAndprivateIP' not in node_info_dict.keys():
        node_info_dict['publicAndprivateIP'] = {}

    node_info_dict['publicAndprivateIP'][index] = item 

node_info_dict

for item in node_info_dict['publicAndprivateIP']:
    print(node_info_dict['publicAndprivateIP'][item])

with open('replicas.txt', 'w') as f:

    for item in node_info_dict['publicAndprivateIP']:

        if (node_info_dict['ServerOrClientregion'][item] ==0):
            f.write("%s\n" % node_info_dict['publicAndprivateIP'][item])


node_info_dict['ServerOrClientregion']

! cat replicas.txt

with open('clients.txt', 'w') as f:

    for item in node_info_dict['publicIP']:

        if (node_info_dict['ServerOrClientregion'][item] ==1):
            f.write("%s\n" % node_info_dict['publicIP'][item])

os.system('cat clients.txt')

node_info_dict

def setup_remote(i):

    ip = node_info_dict['publicIP'][i] 
    if ip!='None':
        command = 'gcloud compute ssh --zone "'+str(node_info_dict['region'][i].strip()[1:-1])+'" "resdb-'+f"{i:03}"+'" --project "ucr-ursa-major-lesani-lab" --command "sudo apt -y update;sudo apt -y install build-essential; sudo apt -y install autoconf libtool libssl-dev libuv1-dev cmake pkg-config cmake-data make; sudo rm -r hs; git clone https://github.com/tejas-shivanand-mane/O2PRBC.git;"'

        print(command)

        output = os.system(command)
        print(output)


def setup_remote2(i):

    ip = node_info_dict['publicIP'][i] 
    if ip!='None':
        current = (ip.split('.'))



        command = 'gcloud compute ssh --zone "'+str(node_info_dict['region'][i].strip()[1:-1])+'" "resdb-'+f"{i:03}"+'" --project "ucr-ursa-major-lesani-lab" --command "cd O2PRBC; git pull;"'

        print(command)

        output = os.system(command)
        print(output)


def git_pull_remote(i):

    ip = node_info_dict['publicIP'][i] 
    if ip!='None':
        current = (ip.split('.'))
        command = 'ssh -o StrictHostKeyChecking=no -i /home/tejas/Documents/'+node_info_dict['key_file'][i].strip()[1:-1]+' '+'ubuntu@ec2-'+str(current[0])+'-'\
                      +str(current[1])+'-'+str(current[2])+'-'+str(current[3])+'.'+node_info_dict['region'][i].strip()[1:-1]+'.compute.amazonaws.com -T "cd O2PRBC; sudo git pull;"'


        output = subprocess.getoutput(command)
        print(output)


def kill_nodes_remote(i):
    ip = node_info_dict['publicIP'][i] 

    if ip!='None':
        current = (ip.split('.'))

        command = 'gcloud compute ssh --zone "'+str(node_info_dict['region'][i].strip()[1:-1])+'" "resdb-'+f"{i:03}"+'" --project "ucr-ursa-major-lesani-lab" --command "cd O2PRBC; sudo killall hotstuff-app; sudo killall hotstuff-client;"'


        output = subprocess.getoutput(command)
        print(output)



def clean_nodes_remote(i):
    ip = node_info_dict['publicIP'][i] 

    if ip!='None':
        current = (ip.split('.'))

        command = 'gcloud compute ssh --zone "'+str(node_info_dict['region'][i].strip()[1:-1])+'" "resdb-'+f"{i:03}"+'" --project "ucr-ursa-major-lesani-lab" --command "sudo rm -f nohup.out;"'


        print(command)
        os.system(command)

experiment = 'test'
os.system('rm hotstuff.gen*')
os.system('echo 3108 | sudo -S sh job_mini.sh '+ experiment)


os.system('cp hotstuff.gen* ../../')


{'node_id': {0: 0, 1: 1, 2: 2, 3: 3, 4: 4, 5: 5, 6: 6, 7: 7, 8: 8, 9: 9, 10: 10, 11: 11, 12: 12, 13: 13, 14: 14, 15: 15, 16: 16, 17: 17, 18: 18, 19: 19, 20: 20, 21: 21, 22: 22, 23: 23, 24: 24, 25: 25, 26: 26, 27: 27, 28: 28, 29: 29, 30: 30, 31: 31, 32: 32, 33: 33, 34: 34, 35: 35, 36: 36, 37: 37, 38: 38, 39: 39, 40: 40, 41: 41, 42: 42, 43: 43, 44: 44, 45: 45, 46: 46, 47: 47, 48: 48}, 'region': {0: " 'europe-west3-c'", 1: " 'europe-west3-c'", 2: " 'europe-west3-c'", 3: " 'europe-west3-c'", 4: " 'europe-west3-c'", 5: " 'europe-west3-c'", 6: " 'europe-west3-c'", 7: " 'europe-west3-c'", 8: " 'europe-west3-c'", 9: " 'europe-west3-c'", 10: " 'europe-west3-c'", 11: " 'europe-west3-c'", 12: " 'europe-west3-c'", 13: " 'europe-west3-c'", 14: " 'europe-west3-c'", 15: " 'europe-west3-c'", 16: " 'europe-west3-c'", 17: " 'europe-west3-c'", 18: " 'europe-west3-c'", 19: " 'europe-west3-c'", 20: " 'europe-west3-c'", 21: " 'europe-west3-c'", 22: " 'europe-west3-c'", 23: " 'europe-west3-c'", 24: " 'europe

ERROR: (gcloud.compute.instances.create) Could not fetch resource:
 - The resource 'projects/ucr-ursa-major-lesani-lab/zones/europe-west3-c/instances/resdb-029' already exists

ERROR: (gcloud.compute.instances.create) Could not fetch resource:
 - The resource 'projects/ucr-ursa-major-lesani-lab/zones/europe-west3-c/instances/resdb-013' already exists

ERROR: (gcloud.compute.instances.create) Could not fetch resource:
 - The resource 'projects/ucr-ursa-major-lesani-lab/zones/europe-west3-c/instances/resdb-011' already exists

Created [https://www.googleapis.com/compute/v1/projects/ucr-ursa-major-lesani-lab/zones/europe-west3-c/instances/resdb-037].
 - The resource 'projects/ubuntu-os-cloud/global/images/ubuntu-2004-focal-v20230831' is deprecated. A suggested replacement is 'projects/ubuntu-os-cloud/global/images/ubuntu-2004-focal-v20250508a'.



NAME       ZONE            MACHINE_TYPE   PREEMPTIBLE  INTERNAL_IP  EXTERNAL_IP     STATUS
resdb-037  europe-west3-c  e2-standard-2               10.156.0.6   34.141.106.238  RUNNING


Created [https://www.googleapis.com/compute/v1/projects/ucr-ursa-major-lesani-lab/zones/europe-west3-c/instances/resdb-043].
 - The resource 'projects/ubuntu-os-cloud/global/images/ubuntu-2004-focal-v20230831' is deprecated. A suggested replacement is 'projects/ubuntu-os-cloud/global/images/ubuntu-2004-focal-v20250508a'.



NAME       ZONE            MACHINE_TYPE   PREEMPTIBLE  INTERNAL_IP  EXTERNAL_IP     STATUS
resdb-043  europe-west3-c  e2-standard-2               10.156.0.5   35.198.178.200  RUNNING


...

Created [https://www.googleapis.com/compute/v1/projects/ucr-ursa-major-lesani-lab/zones/europe-west3-c/instances/resdb-023].
 - The resource 'projects/ubuntu-os-cloud/global/images/ubuntu-2004-focal-v20230831' is deprecated. A suggested replacement is 'projects/ubuntu-os-cloud/global/images/ubuntu-2004-focal-v20250508a'.



NAME       ZONE            MACHINE_TYPE   PREEMPTIBLE  INTERNAL_IP  EXTERNAL_IP     STATUS
resdb-023  europe-west3-c  e2-standard-2               10.156.0.68  34.159.242.151  RUNNING
10.156.0.77
10.156.0.10
10.156.0.69
10.156.0.82
10.156.0.72
10.156.0.80
10.156.0.104
10.156.0.111
10.156.0.89
10.156.0.81
10.156.0.75
10.156.0.3
10.156.0.93
10.156.0.2
10.156.0.99
10.156.0.73
10.156.0.108
10.156.0.13
10.156.0.98
10.156.0.95
10.156.0.96
10.156.0.91
10.156.0.78
10.156.0.68
10.156.0.101
10.156.0.79
10.156.0.94
10.156.0.90
10.156.0.74
10.156.0.4
10.156.0.97
10.156.0.105
10.156.0.14
10.156.0.102
10.156.0.71
10.156.0.112
10.156.0.7
10.156.0.6
10.156.0.9
10.156.0.83
10.156.0.113
10.156.0.103
10.156.0.106
10.156.0.5
10.156.0.11
10.156.0.100
10.156.0.88
10.156.0.107
10.156.0.109
 
10.156.0.77    10.156.0.77
10.156.0.10    10.156.0.10
10.156.0.69    10.156.0.69
10.156.0.82    10.156.0.82
10.156.0.72    10.156.0.72
10.156.0.80    10.156.0.80
10.156.0.104    10.156.0.104
10.156.0.111    10.156.0.111
1

[sudo] password for tejas: CMake Warning:
  No source or binary directory provided.  Both will be assumed to be the
  same as the current working directory, but note that this warning will
  become a fatal error in future CMake releases.




-- Found OpenSSL: /usr/lib/x86_64-linux-gnu/libcrypto.so (found suitable version "3.0.13", minimum required is "1.1.0")  
-- Found OpenSSL: /usr/lib/x86_64-linux-gnu/libcrypto.so (found version "3.0.13")  
-- Could NOT find Doxygen (missing: DOXYGEN_EXECUTABLE) 
-- Configuring done (0.2s)
-- Generating done (0.1s)
-- Build files have been written to: /home/tejas/work/O2PRBC
[ 21%] Built target libsecp256k1
[ 37%] Built target hotstuff
[ 62%] Built target salticidae
[ 64%] Built target salticidae_shared
[ 67%] Built target salticidae_static
[ 70%] Built target hotstuff_static
[ 72%] Built target hotstuff_shared
[ 78%] Built target hotstuff-app
[ 83%] Built target hotstuff-tls-keygen
[ 89%] Built target test_secp256k1
[ 94%] Built target hotstuff-keygen
[100%] Built target hotstuff-client


0

In [12]:
instance_type = 'e2-small'


redirect = '>'

os.system('gcloud compute instances list  --format="value(networkInterfaces[0].networkIP)" '+\
          redirect+' all_internal_ips')



os.system('gcloud compute instances list --format="value(networkInterfaces[0].networkIP)" '+\
      redirect+' all_external_ips')







!cat all_internal_ips
! echo " " 
# !cat all_external_ips



f = open('all_external_ips', "r")
data1 = [line.strip() for line in f.readlines()]

data1 = [x for x in data1 if x!='None']

f = open('all_internal_ips', "r")
data2 = [line.strip() for line in f.readlines()]

data2 = [x for x in data2 if x!='None']



data2, len(data2)

data = []

# for i in range(len(data1)):
#     if data1[i]!='None':
# #         data.append(data1[i] + '    ' + data2[i])
#         data.append('127.0.0.1'+ '    ' + '127.0.0.1')


skip_instances = 0

for i in range(skip_instances, len(data1)):
    if data1[i]!='None':
        data.append(data1[i] + '    ' + data2[i])
#         data.append('127.0.0.1'+ '    ' + '127.0.0.1')

data, len(data)

for index, item in enumerate(data1):
    if 'publicIP' not in node_info_dict.keys():
        node_info_dict['publicIP'] = {}


    node_info_dict['publicIP'][index] = item 


for index, item in enumerate(data2):
    if 'privateIP' not in node_info_dict.keys():
        node_info_dict['privateIP'] = {}

    node_info_dict['privateIP'][index] = item 



for index, item in enumerate(data):
    if 'publicAndprivateIP' not in node_info_dict.keys():
        node_info_dict['publicAndprivateIP'] = {}

    node_info_dict['publicAndprivateIP'][index] = item 

node_info_dict

for item in node_info_dict['publicAndprivateIP']:
    print(node_info_dict['publicAndprivateIP'][item])

with open('replicas.txt', 'w') as f:

    for item in node_info_dict['publicAndprivateIP']:

        if (node_info_dict['ServerOrClientregion'][item] ==0):
            f.write("%s\n" % node_info_dict['publicAndprivateIP'][item])


node_info_dict['ServerOrClientregion']

! cat replicas.txt

10.156.0.77
10.156.0.10
10.156.0.69
10.156.0.82
10.156.0.72
10.156.0.80
10.156.0.104
10.156.0.111
10.156.0.89
10.156.0.81
10.156.0.75
10.156.0.3
10.156.0.93
10.156.0.2
10.156.0.99
10.156.0.73
10.156.0.108
10.156.0.13
10.156.0.98
10.156.0.95
10.156.0.96
10.156.0.91
10.156.0.78
10.156.0.68
10.156.0.101
10.156.0.79
10.156.0.94
10.156.0.90
10.156.0.74
10.156.0.4
10.156.0.97
10.156.0.105
10.156.0.14
10.156.0.102
10.156.0.71
10.156.0.112
10.156.0.7
10.156.0.6
10.156.0.9
10.156.0.83
10.156.0.113
10.156.0.103
10.156.0.106
10.156.0.5
10.156.0.11
10.156.0.100
10.156.0.88
10.156.0.107
10.156.0.109
 
10.156.0.77    10.156.0.77
10.156.0.10    10.156.0.10
10.156.0.69    10.156.0.69
10.156.0.82    10.156.0.82
10.156.0.72    10.156.0.72
10.156.0.80    10.156.0.80
10.156.0.104    10.156.0.104
10.156.0.111    10.156.0.111
10.156.0.89    10.156.0.89
10.156.0.81    10.156.0.81
10.156.0.75    10.156.0.75
10.156.0.3    10.156.0.3
10.156.0.93    10.156.0.93
10.156.0.2    10.156.0.2
10.156.0.99    10.156.0.99

In [13]:
with open('clients.txt', 'w') as f:

    for item in node_info_dict['publicIP']:

        if (node_info_dict['ServerOrClientregion'][item] ==1):
            f.write("%s\n" % node_info_dict['publicIP'][item])

os.system('cat clients.txt')

node_info_dict

def setup_remote(i):

    ip = node_info_dict['publicIP'][i] 
    if ip!='None':
        command = 'gcloud compute ssh --zone "'+str(node_info_dict['region'][i].strip()[1:-1])+'" "resdb-'+f"{i:03}"+'" --project "ucr-ursa-major-lesani-lab" --command "sudo apt -y update;sudo apt -y install build-essential; sudo apt -y install autoconf libtool libssl-dev libuv1-dev cmake pkg-config cmake-data make; sudo rm -r hs; git clone https://github.com/tejas-shivanand-mane/O2PRBC.git;"'

        print(command)

        output = os.system(command)
        print(output)


def setup_remote2(i):

    ip = node_info_dict['publicIP'][i] 
    if ip!='None':
        current = (ip.split('.'))



        command = 'gcloud compute ssh --zone "'+str(node_info_dict['region'][i].strip()[1:-1])+'" "resdb-'+f"{i:03}"+'" --project "ucr-ursa-major-lesani-lab" --command "cd O2PRBC; git pull;"'

        print(command)

        output = os.system(command)
        print(output)


def git_pull_remote(i):

    ip = node_info_dict['publicIP'][i] 
    if ip!='None':
        current = (ip.split('.'))
        command = 'ssh -o StrictHostKeyChecking=no -i /home/tejas/Documents/'+node_info_dict['key_file'][i].strip()[1:-1]+' '+'ubuntu@ec2-'+str(current[0])+'-'\
                      +str(current[1])+'-'+str(current[2])+'-'+str(current[3])+'.'+node_info_dict['region'][i].strip()[1:-1]+'.compute.amazonaws.com -T "cd O2PRBC; sudo git pull;"'


        output = subprocess.getoutput(command)
        print(output)





def kill_nodes_remote(i):
    ip = node_info_dict['publicIP'][i] 

    if ip!='None':
        current = (ip.split('.'))

        command = 'gcloud compute ssh --zone "'+str(node_info_dict['region'][i].strip()[1:-1])+'" "resdb-'+f"{i:03}"+'" --project "ucr-ursa-major-lesani-lab" --command "cd O2PRBC; sudo killall hotstuff-app; sudo killall hotstuff-client;"'


        output = subprocess.getoutput(command)
        print(output)



def clean_nodes_remote(i):
    ip = node_info_dict['publicIP'][i] 

    if ip!='None':
        current = (ip.split('.'))

        command = 'gcloud compute ssh --zone "'+str(node_info_dict['region'][i].strip()[1:-1])+'" "resdb-'+f"{i:03}"+'" --project "ucr-ursa-major-lesani-lab" --command "sudo rm -f nohup.out;"'


        print(command)
        os.system(command)

10.156.0.109


In [14]:
experiment = 'test'
os.system('rm hotstuff.gen*')
os.system('echo 3108 | sudo -S sh job_mini.sh '+ experiment)

os.system('cp hotstuff.gen* ../../')

os.system('cd ../../; git add .; git commit -m "test"; git push -f origin HEAD:main')

Name of TestRun: test
-- Found OpenSSL: /usr/lib/x86_64-linux-gnu/libcrypto.so (found suitable version "3.0.13", minimum required is "1.1.0")  
-- Found OpenSSL: /usr/lib/x86_64-linux-gnu/libcrypto.so (found version "3.0.13")  


[sudo] password for tejas: CMake Warning:
  No source or binary directory provided.  Both will be assumed to be the
  same as the current working directory, but note that this warning will
  become a fatal error in future CMake releases.




-- Could NOT find Doxygen (missing: DOXYGEN_EXECUTABLE) 
-- Configuring done (0.1s)
-- Generating done (0.0s)
-- Build files have been written to: /home/tejas/work/O2PRBC
[ 21%] Built target libsecp256k1
[ 37%] Built target hotstuff
[ 62%] Built target salticidae
[ 64%] Built target salticidae_shared
[ 67%] Built target salticidae_static
[ 70%] Built target hotstuff_shared
[ 72%] Built target hotstuff_static
[ 78%] Built target hotstuff-tls-keygen
[ 83%] Built target hotstuff-keygen
[ 89%] Built target test_secp256k1
[ 94%] Built target hotstuff-app
[100%] Built target hotstuff-client
[main d9d6277] test
 49 files changed, 192 insertions(+), 192 deletions(-)


To github.com:tejas-shivanand-mane/O2PRBC.git
   554638d..d9d6277  HEAD -> main


0

In [15]:
prefix = 'O2PRBC_single_collect_'


In [19]:
# if run_opt !='r':
#     time.sleep(150)

run_opt ='n'

In [20]:
results = Parallel(n_jobs=60)(delayed(kill_nodes_remote)(i)\
                                                                       for i in node_info_dict['publicIP'])


# for i in node_info_dict['publicIP']:
#     print(i)
#     kill_nodes_remote(i)


if run_opt == 'r':
    results = Parallel(n_jobs=60)(delayed(setup_remote2)(i)\
                                                                       for i in node_info_dict['node_id'])

else:
    results = Parallel(n_jobs=60)(delayed(setup_remote)(i)\
                                                                       for i in node_info_dict['node_id'])


print(results)  

Hit:1 http://europe-west3.gce.archive.ubuntu.com/ubuntu focal InRelease
Hit:2 http://europe-west3.gce.archive.ubuntu.com/ubuntu focal-updates InRelease
Hit:3 http://security.ubuntu.com/ubuntu focal-security InRelease
Hit:4 http://europe-west3.gce.archive.ubuntu.com/ubuntu focal-backports InRelease


Hit:1 http://europe-west3.gce.archive.ubuntu.com/ubuntu focal InRelease
Hit:2 http://europe-west3.gce.archive.ubuntu.com/ubuntu focal-updates InRelease
Hit:3 http://security.ubuntu.com/ubuntu focal-security InRelease
Hit:4 http://europe-west3.gce.archive.ubuntu.com/ubuntu focal-backports InRelease


Reading package lists...Hit:1 http://europe-west3.gce.archive.ubuntu.com/ubuntu focal InRelease
Hit:2 http://europe-west3.gce.archive.ubuntu.com/ubuntu focal-updates InRelease
Get:3 http://security.ubuntu.com/ubuntu focal-security InRelease [128 kB]
Hit:4 http://europe-west3.gce.archive.ubuntu.com/ubuntu focal-backports InRelease


Hit:1 http://europe-west3.gce.archive.ubuntu.com/ubuntu focal InRelease
Hit:2 http://europe-west3.gce.archive.ubuntu.com/ubuntu focal-updates InRelease
Hit:3 http://europe-west3.gce.archive.ubuntu.com/ubuntu focal-backports InRelease
Get:4 http://security.ubuntu.com/ubuntu focal-security InRelease [128 kB]
Hit:1 http://europe-west3.gce.archive.ubuntu.com/ubuntu focal InRelease
Hit:2 http://europe-west3.gce.archive.ubuntu.com/ubuntu focal-updates InRelease
Hit:3 http://europe-west3.gce.archive.ubuntu.com/ubuntu focal-backports InRelease
Hit:4 http://security.ubuntu.com/ubuntu focal-security InRelease
Hit:1 http://europe-west3.gce.archive.ubuntu.com/ubuntu focal InRelease
Hit:2 http://europe-west3.gce.archive.ubuntu.com/ubuntu focal-updates InRelease
Hit:3 http://europe-west3.gce.archive.ubuntu.com/ubuntu focal-backports InRelease
Hit:4 http://security.ubuntu.com/ubuntu focal-security InRelease
Reading package lists...Fetched 128 kB in 1s (186 kB/s)
Reading package lists...Hit:1 http://e

Hit:1 http://europe-west3.gce.archive.ubuntu.com/ubuntu focal InRelease
Hit:2 http://europe-west3.gce.archive.ubuntu.com/ubuntu focal-updates InRelease
Hit:3 http://security.ubuntu.com/ubuntu focal-security InRelease
Hit:1 http://europe-west3.gce.archive.ubuntu.com/ubuntu focal InRelease
Hit:2 http://europe-west3.gce.archive.ubuntu.com/ubuntu focal-updates InRelease
Hit:3 http://security.ubuntu.com/ubuntu focal-security InRelease
Hit:4 http://europe-west3.gce.archive.ubuntu.com/ubuntu focal-backports InRelease
Hit:4 http://europe-west3.gce.archive.ubuntu.com/ubuntu focal-backports InRelease
Hit:1 http://europe-west3.gce.archive.ubuntu.com/ubuntu focal InRelease
Hit:2 http://europe-west3.gce.archive.ubuntu.com/ubuntu focal-updates InRelease
Hit:3 http://europe-west3.gce.archive.ubuntu.com/ubuntu focal-backports InRelease
Hit:1 http://europe-west3.gce.archive.ubuntu.com/ubuntu focal InRelease
Hit:2 http://europe-west3.gce.archive.ubuntu.com/ubuntu focal-updates InRelease
Hit:1 http://eur

...

Reading package lists...
Building dependency tree...
Reading state information...
make is already the newest version (4.2.1-1.2).
make set to manually installed.
The following packages were automatically installed and are no longer required:
  libatasmart4 libblockdev-fs2 libblockdev-loop2 libblockdev-part-err2
  libblockdev-part2 libblockdev-swap2 libblockdev-utils2 libblockdev2
  libmbim-glib4 libmbim-proxy libmm-glib0 libnspr4 libnss3 libnuma1
  libparted-fs-resize0 libqmi-glib5 libqmi-proxy libudisks2-0 libxmlb2
  usb-modeswitch usb-modeswitch-data
Use 'sudo apt autoremove' to remove them.
The following additional packages will be installed:
  automake autotools-dev libarchive13 libjsoncpp1 libltdl-dev librhash0
  libssl1.1 libuv1 m4
Suggested packages:
  autoconf-archive gnu-standards autoconf-doc gettext cmake-doc ninja-build
  lrzip libtool-doc libssl-doc gfortran | fortran95-compiler gcj-jdk m4-doc
The following NEW packages will be installed:
  autoconf automake autotools-dev 

debconf: unable to initialize frontend: Dialog
debconf: (Dialog frontend will not work on a dumb terminal, an emacs shell buffer, or without a controlling terminal.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 


Fetched 10.3 MB in 0s (48.3 MB/s)
(Reading database ... 67840 files and directories currently installed.)
Preparing to unpack .../00-libssl1.1_1.1.1f-1ubuntu2.24_amd64.deb ...
Unpacking libssl1.1:amd64 (1.1.1f-1ubuntu2.24) over (1.1.1f-1ubuntu2.19) ...
Preparing to unpack .../01-libuv1_1.34.2-1ubuntu1.5_amd64.deb ...
Unpacking libuv1:amd64 (1.34.2-1ubuntu1.5) over (1.34.2-1ubuntu1.3) ...
Selecting previously unselected package m4.
Preparing to unpack .../02-m4_1.4.18-4_amd64.deb ...
Unpacking m4 (1.4.18-4) ...
Selecting previously unselected package autoconf.
Preparing to unpack .../03-autoconf_2.69-11.1_all.deb ...
Unpacking autoconf (2.69-11.1) ...
Selecting previously unselected package autotools-dev.
Preparing to unpack .../04-autotools-dev_20180224.1_all.deb ...
Unpacking autotools-dev (20180224.1) ...
Selecting previously unselected package automake.
Preparing to unpack .../05-automake_1%3a1.16.1-4ubuntu6_all.deb ...
Unpacking automake (1:1.16.1-4ubuntu6) ...
Selecting previously

rm: cannot remove 'hs': No such file or directory
Cloning into 'O2PRBC'...


[None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None]


In [21]:
# for i in node_info_dict['publicIP']:
#     print(i)
#     setup_remote2(i)

# time.process_time()



def compile_remote(i):

    ip = node_info_dict['publicIP'][i] 
    if ip!='None':
        current = (ip.split('.'))


        command = 'gcloud compute ssh --zone "'+str(node_info_dict['region'][i].strip()[1:-1])+'" "resdb-'+f"{i:03}"+'" --project "ucr-ursa-major-lesani-lab" --command "cd O2PRBC; sudo nohup sh compile_job.sh >../make_job.out 2>&1 &"'


        print(command)
        output = os.system(command)
        print(output)







results = Parallel(n_jobs=60)(delayed(compile_remote)(i)\
                                                                       for i in list(node_info_dict['node_id'].keys()))


print(results)  


t_compile =  0

if run_opt=='r':
    t_compile = 60
else:
    t_compile = 150

t_compile

time.sleep(t_compile)

[None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None]


In [22]:
results = Parallel(n_jobs=60)(delayed(kill_nodes_remote)(i)\
                                                                       for i in node_info_dict['publicIP'])



results = Parallel(n_jobs=60 )(delayed(clean_nodes_remote)(i)\
                                                                       for i in node_info_dict['node_id'])


print(results)  


# node_info_dict

def run_server_node_remote(i):
    ip = node_info_dict['publicIP'][i] 
#     if ip!='None' and( ip in nodeIps[:-3]):

    if ip!='None' and node_info_dict['ServerOrClientregion'][i]==0:

        current = (ip.split('.'))


        command = 'gcloud compute ssh --zone "'+str(node_info_dict['region'][i].strip()[1:-1])+'" "resdb-'+f"{i:03}"+'" --project "ucr-ursa-major-lesani-lab" --command "cd O2PRBC; sudo nohup ./examples/hotstuff-app --conf hotstuff.gen-sec'+str(node_info_dict['ServerID'][i])+'.conf >../nohup.out 2>&1 &"'

        print(i,command)
        os.system(command)


results = Parallel(n_jobs=60)(delayed(run_server_node_remote)(i)\
for i in node_info_dict['node_id'])

print(results)  
time.sleep(45)

[None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None]
[None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None]


In [23]:
def run_client_node_remote(i):
    ip = node_info_dict['publicIP'][i] 

    if ip!='None' and node_info_dict['ServerOrClientregion'][i]==1:

        current = (ip.split('.'))


        command = 'gcloud compute ssh --zone "'+str(node_info_dict['region'][i].strip()[1:-1])+'" "resdb-'+f"{i:03}"+'" --project "ucr-ursa-major-lesani-lab" --command "cd O2PRBC; sudo nohup ./examples/hotstuff-client --idx '+str(node_info_dict['clientID'][i])+' --iter -1 --max-async 400 >>../nohup.out 2>&1 &"'





        print(command)
        os.system(command)



#         results = Parallel(n_jobs=60)(delayed(kill_nodes_remote)(i)\
#                            for i in [17])



results = Parallel(n_jobs=60)(delayed(run_client_node_remote)(i)\
               for i in node_info_dict['node_id'])
time.sleep(80)
#         results = Parallel(n_jobs=60)(delayed(kill_nodes_remote)(i)\
#                                    for i in [18,19])


print(results)  



# results = Parallel(n_jobs=60)(delayed(kill_nodes_remote)(i)\
#                                                                        for i in [4])


time.sleep(20)



# def run_client_node_remote(i):
#     ip = node_info_dict['publicIP'][i] 

#     if ip!='None' and node_info_dict['ServerOrClientregion'][i]==1:

#         current = (ip.split('.'))


#         command = 'gcloud compute ssh --zone "'+str(node_info_dict['region'][i].strip()[1:-1])+'" "resdb-'+f"{i:03}"+'" --project "ucr-ursa-major-lesani-lab" --command "cd O2PRBC; sudo nohup ./examples/hotstuff-client --idx '+str(node_info_dict['clientID'][i])+' --iter -1 --max-async 100 >>../nohup1.out 2>&1 &"'





#         print(command)
#         os.system(command)



# #         results = Parallel(n_jobs=60)(delayed(kill_nodes_remote)(i)\
# #                            for i in [17])



# results = Parallel(n_jobs=60)(delayed(run_client_node_remote)(i)\
#                for i in node_info_dict['node_id'])
# time.sleep(100)



results = Parallel(n_jobs=60)(delayed(kill_nodes_remote)(i)\
                                                                       for i in node_info_dict['publicIP'])




# for i in node_info_dict['publicIP']:
#     print(i)
#     kill_nodes_remote(i)

[None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None]


In [24]:
os.system('cd ../../../dump_files; echo 3108 | sudo -S  rm *')

#         for i in node_info_dict['publicIP']:
#             ip = node_info_dict['publicIP'][i] 

#             if ip!='None' and node_info_dict['ServerOrClientregion'][i]==1:
#                 current = (ip.split('.'))
#                 command = 'gcloud compute scp --zone "'+str(node_info_dict['region'][i].strip()[1:-1]) +'"'+' resdb-'+f"{i:03}"+':/home/tejas/nohup.out ../../../dump_files/nohup_client_'+str(node_info_dict['clientID'][i])+'.out'
#                 print(command)
#                 os.system(command)

#         break






experiment = cfig

if not os.path.exists('../../../Experiments/'+prefix+experiment):
    os.mkdir('../../../Experiments/'+prefix+experiment)








temp_experiment = '../../dump_files'






os.system('cp -r ../../../dump_files ../../../Experiments/'+prefix+experiment)    



#         os.system('gcloud compute instances delete --zone=europe-west3-c --quiet $(gcloud compute instances list --format="value(name)")')
#         os.system('gcloud compute instances delete --zone=asia-south1-c --quiet $(gcloud compute instances list --format="value(name)")')
#         os.system('gcloud compute instances delete --zone=us-west1-b --quiet $(gcloud compute instances list --format="value(name)")')   

#     except:


#         experiment = cfig

#         if not os.path.exists('../../../Experiments/h_ours'+experiment):
#             os.mkdir('../../../Experiments/h_ours'+experiment)

#         os.system('gcloud compute instances delete --zone=europe-west3-c --quiet $(gcloud compute instances list --format="value(name)")')
#         os.system('gcloud compute instances delete --zone=asia-south1-c --quiet $(gcloud compute instances list --format="value(name)")')
#         os.system('gcloud compute instances delete --zone=us-west1-b --quiet $(gcloud compute instances list --format="value(name)")')

#         continue


for i in node_info_dict['publicIP']:
    ip = node_info_dict['publicIP'][i] 

    if ip!='None' and node_info_dict['ServerOrClientregion'][i]==1:
        current = (ip.split('.'))
        command = 'gcloud compute scp --zone "'+str(node_info_dict['region'][i].strip()[1:-1]) +'"'+' resdb-'+f"{i:03}"+':/home/tejas/nohup.out ../../../dump_files/nohup_client_'+str(node_info_dict['clientID'][i])+'.out'
        os.system(command)
        
        
        command = 'gcloud compute scp --zone "'+str(node_info_dict['region'][i].strip()[1:-1]) +'"'+' resdb-'+f"{i:03}"+':/home/tejas/nohup1.out ../../../dump_files/nohup_client_'+str(1+node_info_dict['clientID'][i])+'.out'
        os.system(command)


os.system('cp -r ../../../dump_files ../../../Experiments/'+prefix+experiment)    

[sudo] password for tejas: 

hotstuff-app: no process found
hotstuff-client: no process found
gcloud compute ssh --zone "europe-west3-c" "resdb-021" --project "ucr-ursa-major-lesani-lab" --command "sudo apt -y update;sudo apt -y install build-essential; sudo apt -y install autoconf libtool libssl-dev libuv1-dev cmake pkg-config cmake-data make; sudo rm -r hs; git clone https://github.com/tejas-shivanand-mane/O2PRBC.git;"
32768
gcloud compute ssh --zone "europe-west3-c" "resdb-038" --project "ucr-ursa-major-lesani-lab" --command "cd O2PRBC; sudo nohup sh compile_job.sh >../make_job.out 2>&1 &"
0
gcloud compute ssh --zone "europe-west3-c" "resdb-005" --project "ucr-ursa-major-lesani-lab" --command "sudo rm -f nohup.out;"
30 gcloud compute ssh --zone "europe-west3-c" "resdb-030" --project "ucr-ursa-major-lesani-lab" --command "cd O2PRBC; sudo nohup ./examples/hotstuff-app --conf hotstuff.gen-sec30.conf >../nohup.out 2>&1 &"
hotstuff-app: no process found
hotstuff-client: no process found
gcloud compute ssh --zone "eu

hotstuff-app: no process found
hotstuff-client: no process found
gcloud compute ssh --zone "europe-west3-c" "resdb-028" --project "ucr-ursa-major-lesani-lab" --command "sudo apt -y update;sudo apt -y install build-essential; sudo apt -y install autoconf libtool libssl-dev libuv1-dev cmake pkg-config cmake-data make; sudo rm -r hs; git clone https://github.com/tejas-shivanand-mane/O2PRBC.git;"
32768
gcloud compute ssh --zone "europe-west3-c" "resdb-014" --project "ucr-ursa-major-lesani-lab" --command "cd O2PRBC; sudo nohup sh compile_job.sh >../make_job.out 2>&1 &"
0
hotstuff-app: no process found
hotstuff-client: no process found
gcloud compute ssh --zone "europe-west3-c" "resdb-034" --project "ucr-ursa-major-lesani-lab" --command "sudo rm -f nohup.out;"
40 gcloud compute ssh --zone "europe-west3-c" "resdb-040" --project "ucr-ursa-major-lesani-lab" --command "cd O2PRBC; sudo nohup ./examples/hotstuff-app --conf hotstuff.gen-sec40.conf >../nohup.out 2>&1 &"
gcloud compute ssh --zone "eu

hotstuff-app: no process found
hotstuff-client: no process found
gcloud compute ssh --zone "europe-west3-c" "resdb-030" --project "ucr-ursa-major-lesani-lab" --command "sudo apt -y update;sudo apt -y install build-essential; sudo apt -y install autoconf libtool libssl-dev libuv1-dev cmake pkg-config cmake-data make; sudo rm -r hs; git clone https://github.com/tejas-shivanand-mane/O2PRBC.git;"
32768
gcloud compute ssh --zone "europe-west3-c" "resdb-019" --project "ucr-ursa-major-lesani-lab" --command "cd O2PRBC; sudo nohup sh compile_job.sh >../make_job.out 2>&1 &"
0
hotstuff-app: no process found
hotstuff-client: no process found
gcloud compute ssh --zone "europe-west3-c" "resdb-038" --project "ucr-ursa-major-lesani-lab" --command "sudo rm -f nohup.out;"
hotstuff-client: no process found
hotstuff-app: no process found
hotstuff-client: no process found
gcloud compute ssh --zone "europe-west3-c" "resdb-027" --project "ucr-ursa-major-lesani-lab" --command "sudo apt -y update;sudo apt -y i

hotstuff-app: no process found
hotstuff-client: no process found
gcloud compute ssh --zone "europe-west3-c" "resdb-025" --project "ucr-ursa-major-lesani-lab" --command "sudo apt -y update;sudo apt -y install build-essential; sudo apt -y install autoconf libtool libssl-dev libuv1-dev cmake pkg-config cmake-data make; sudo rm -r hs; git clone https://github.com/tejas-shivanand-mane/O2PRBC.git;"
32768
gcloud compute ssh --zone "europe-west3-c" "resdb-018" --project "ucr-ursa-major-lesani-lab" --command "cd O2PRBC; sudo nohup sh compile_job.sh >../make_job.out 2>&1 &"
0
hotstuff-app: no process found
hotstuff-client: no process found
gcloud compute ssh --zone "europe-west3-c" "resdb-048" --project "ucr-ursa-major-lesani-lab" --command "sudo rm -f nohup.out;"
38 gcloud compute ssh --zone "europe-west3-c" "resdb-038" --project "ucr-ursa-major-lesani-lab" --command "cd O2PRBC; sudo nohup ./examples/hotstuff-app --conf hotstuff.gen-sec38.conf >../nohup.out 2>&1 &"
hotstuff-client: no process f

hotstuff-app: no process found
hotstuff-client: no process found
gcloud compute ssh --zone "europe-west3-c" "resdb-007" --project "ucr-ursa-major-lesani-lab" --command "cd O2PRBC; sudo nohup sh compile_job.sh >../make_job.out 2>&1 &"
0
hotstuff-app: no process found
hotstuff-client: no process found
gcloud compute ssh --zone "europe-west3-c" "resdb-020" --project "ucr-ursa-major-lesani-lab" --command "sudo rm -f nohup.out;"
27 gcloud compute ssh --zone "europe-west3-c" "resdb-027" --project "ucr-ursa-major-lesani-lab" --command "cd O2PRBC; sudo nohup ./examples/hotstuff-app --conf hotstuff.gen-sec27.conf >../nohup.out 2>&1 &"
hotstuff-client: no process found
gcloud compute ssh --zone "europe-west3-c" "resdb-005" --project "ucr-ursa-major-lesani-lab" --command "sudo apt -y update;sudo apt -y install build-essential; sudo apt -y install autoconf libtool libssl-dev libuv1-dev cmake pkg-config cmake-data make; sudo rm -r hs; git clone https://github.com/tejas-shivanand-mane/O2PRBC.git;"
3

gcloud compute ssh --zone "europe-west3-c" "resdb-000" --project "ucr-ursa-major-lesani-lab" --command "sudo apt -y update;sudo apt -y install build-essential; sudo apt -y install autoconf libtool libssl-dev libuv1-dev cmake pkg-config cmake-data make; sudo rm -r hs; git clone https://github.com/tejas-shivanand-mane/O2PRBC.git;"
32768
gcloud compute ssh --zone "europe-west3-c" "resdb-042" --project "ucr-ursa-major-lesani-lab" --command "cd O2PRBC; sudo nohup sh compile_job.sh >../make_job.out 2>&1 &"
0
gcloud compute ssh --zone "europe-west3-c" "resdb-008" --project "ucr-ursa-major-lesani-lab" --command "sudo rm -f nohup.out;"
23 gcloud compute ssh --zone "europe-west3-c" "resdb-023" --project "ucr-ursa-major-lesani-lab" --command "cd O2PRBC; sudo nohup ./examples/hotstuff-app --conf hotstuff.gen-sec23.conf >../nohup.out 2>&1 &"
hotstuff-client: no process found
hotstuff-app: no process found
hotstuff-client: no process found
gcloud compute ssh --zone "europe-west3-c" "resdb-005" --pro

/usr/bin/scp: /home/tejas/nohup1.out: No such file or directory
ERROR: (gcloud.compute.scp) [/usr/bin/scp] exited with return code [1].


0

In [25]:

for i in node_info_dict['publicIP']:
    ip = node_info_dict['publicIP'][i] 

    if ip!='None' and node_info_dict['ServerOrClientregion'][i]==0:
        current = (ip.split('.'))
        command = 'gcloud compute scp --zone "'+str(node_info_dict['region'][i].strip()[1:-1]) +'"'+' resdb-'+f"{i:03}"+':/home/tejas/nohup.out ../../../dump_files/nohup_server_'+str(node_info_dict['ServerID'][i])+'.out'
#         substrings = ['_1.', '_5.', '_9.', '_13.', '_17.', '_21.']
#         if any(substring in command for substring in substrings):

#             print(command)

        os.system(command)
        break


os.system('cp -r ../../../dump_files ../../../Experiments/'+prefix+experiment)  

client_loop: send disconnect: Broken pipe
/usr/bin/scp: Connection closed
ERROR: (gcloud.compute.scp) [/usr/bin/scp] exited with return code [255].


0

In [26]:
'../../../Experiments/'+prefix+experiment

'../../../Experiments/O2PRBC_single_collect_test48'